### Environment Set Up

In [ ]:
!git clone -b r main https://github.com/aladenisun/MSAAI_530_FinalProject

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install -q -r /content/MSAAI_530_FinalProject/requirements.txt

In [ ]:
import sys
import os

def in_colab():
    return "COLAB_GPU" in os.environ or "google.colab" in sys.modules

if in_colab():
    # Running in Google Colab
    repo_path = "/content/coral-reef-classification"
    data_path = "/content/coral-reef-classification/data/processed"

    # Set working directory to the repo root
    os.chdir(repo_path)

else:
    # Running locally in VS Code
    repo_path = os.path.abspath(os.path.join(os.getcwd(), ".."))
    data_path = os.path.abspath(os.path.join(repo_path, "data", "processed"))

    # Add repo root to Python path
    if repo_path not in sys.path:
        sys.path.append(repo_path)

    # Set working directory to the repo root
    os.chdir(repo_path)

print("Using repo path:", repo_path)
print("Using data path:", data_path)
print("CWD:", os.getcwd())

# LSTM Model

This Long Short-Term Memory (LSTM) deep learning model is designed to learn and classify patterns in indoor air quality dynamics rather than identifying specific substances. Using time series data from three gas sensors along with temperature and humidity, the model captures how air conditions evolve over time during normal household activity and during events that degrade air quality, such as poor ventilation, cooking, or the presence of harmful gases. By modeling temporal dependencies, the LSTM can distinguish brief, harmless fluctuations from sustained or abnormal changes that may impact occupant health.

The model runs in the cloud and processes incoming data in near real time, producing air quality state predictions that are smoothed over multiple windows before triggering alerts. Model performance is evaluated using classification metrics such as accuracy and F1-score, and results are visualized as predicted air quality states over time. 


In [ ]:
# Import Libraries
import numpy as np
import pandas as pd